In [1]:
import os
import cv2

In [2]:
features=[]
targets=[]
for x in range (0,43):
    Imagelist=os.listdir("E:/datasets/traffic signs" + "/" + str(x))
    for y in Imagelist:
        imagearr=cv2.imread("E:/datasets/traffic signs" + "/" + str(x) + "/" + y)
        features.append(imagearr)
        targets.append(x)
    


In [3]:
import numpy as np
features = np.array(features)
targets = np.array(targets)

In [4]:
features.shape

(73139, 32, 32, 3)

In [5]:
from sklearn.model_selection import train_test_split
features_train,features_test,target_train,targets_test=train_test_split(features,targets,test_size=0.2)

In [7]:
def preprocessing(image):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = image/255
    return image

In [7]:
features_train=np.array(list(map(preprocessing,features_train)))

In [8]:
features_train.shape

(58511, 32, 32)

In [10]:
features_train = features_train.reshape(58511,32,32,1)

In [11]:
from keras.preprocessing.image import ImageDataGenerator
dataGen=ImageDataGenerator(rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1)
dataGen.fit(features_train)
dataGen.flow(features_train,target_train,batch_size=20)

In [ ]:
#step1

In [12]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential
model = Sequential()
model.add(Conv2D(60,(3,3),activation="relu",input_shape=(32,32,1)))
model.add(Conv2D(60,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(60,(3,3),activation="relu"))
model.add(Conv2D(60,(3,3),activation="relu"))
model.add(Conv2D(60,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(43,activation="softmax"))

In [ ]:
#step2

In [13]:
from tensorflow.keras.optimizers import Adam

In [14]:
model.compile(optimizer="Adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [ ]:
#step3

In [15]:
model.fit_generator(dataGen.flow(features_train,target_train,batch_size=20),epochs=5)

E:\Anaconda\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
2926/2926 [==============================] - 343s 116ms/step - loss: 1.3221 - accuracy: 0.6214
Epoch 2/5
2926/2926 [==============================] - 359s 123ms/step - loss: 0.2588 - accuracy: 0.9197
Epoch 3/5
2926/2926 [==============================] - 403s 138ms/step - loss: 0.1737 - accuracy: 0.9461
Epoch 4/5
2926/2926 [==============================] - 399s 136ms/step - loss: 0.1393 - accuracy: 0.9574
Epoch 5/5
2926/2926 [==============================] - 417s 143ms/step - loss: 0.1238 - accuracy: 0.9622


In [2]:
from keras.models import model_from_json
model_json = model.to_json()
with open ("verzeoAItraffic.json", "w" ) as abc:
    abc.write (model_json)
    abc.close()
    model.save_weights("verzeoAItrafficWeights.h5")
    print("Save the model")

NameError: name 'model' is not defined

In [3]:
json_file=open("verzeoAItraffic.json","r")
loaded_model_json=json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("verzeoAItrafficWeights.h5")

In [4]:
import cv2
import numpy as np
capt=cv2.VideoCapture(0)
capt.set(3,640)
capt.set(4,480)
capt.set(10,180)


False

In [8]:
while True:
    message,image=capt.read()
    imagearr = np.asarray(image)
    imagearr=cv2.resize(imagearr,(32,32))
    imagearr=preprocessing(imagearr)
    imagearr=imagearr.reshape(1,32,32,1)
    prediction=loaded_model.predict(imagearr)
    classIndex = np.argmax(loaded_model.predict(imagearr), axis=-1)
    cv2.putText(image,"Class: ",(20,35),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    cv2.imshow("model prediction",image)
    returnedValue=cv2.waitKey(1)
    if returnedValue==ord("s") or ord("S"):
        cv2.destroyAllWindows
        break